# Simulate Execution (TWAP / VWAP)
Loads `data/fake_market_data.csv`, applies TWAP (default) or VWAP to example signals, and writes `outputs/ExecutionResults.csv`.


In [5]:
from pathlib import Path
import csv, math

base = Path().resolve()
market_csv = base / 'data' / 'fake_market_data.csv'
output_csv = base / 'outputs' / 'ExecutionResults.csv'

def load_market_data(path: Path):
    data = {}
    with path.open('r', newline='') as f:
        r = csv.DictReader(f)
        for row in r:
            row = dict(row)
            row['Volume'] = int(float(row['Volume']))
            row['Price'] = float(row['Price'])
            t = row['Ticker'].strip().upper()
            data.setdefault(t, []).append(row)
    for t in data:
        data[t].sort(key=lambda x: x['Date'])
    return data

def save_execution_results(path: Path, rows):
    path.parent.mkdir(parents=True, exist_ok=True)
    fields = ['Ticker','Action','RequestedQty','ExecutedQty','AvgFillPrice','FinalPrice','PNL']
    with path.open('w', newline='') as f:
        w = csv.DictWriter(f, fieldnames=fields)
        w.writeheader(); w.writerows(rows)

def slice_twap(total_qty, n):
    if n <= 0: return []
    per = total_qty // n; rem = total_qty - per*n
    return [per + (1 if i < rem else 0) for i in range(n)]

def slice_vwap(total_qty, vols):
    if total_qty <= 0 or not vols: return [0]*len(vols)
    tot = max(1, sum(max(0,v) for v in vols))
    raw = [total_qty*(max(0,v)/tot) for v in vols]
    flo = [int(math.floor(x)) for x in raw]
    rem = total_qty - sum(flo)
    order = sorted(range(len(vols)), key=lambda i: raw[i]-flo[i], reverse=True)
    for i in range(rem):
        flo[order[i % len(vols)]] += 1
    return flo

def cap_by_participation(qtys, vols, max_participation=1.0):
    return [min(q, int(math.floor(max_participation*max(0,v)))) for q,v in zip(qtys, vols)]

def avg_fill_price(prices, fills):
    notional = sum(p*q for p,q in zip(prices, fills))
    qty = sum(fills)
    return (notional/qty) if qty>0 else 0.0

def pnl(side, avg_fill, final_price, qty):
    side = side.upper()
    return (final_price-avg_fill)*qty if side=='BUY' else (avg_fill-final_price)*qty

def simulate_execution(signals, market, method='TWAP', max_participation=1.0, reference_price_by='first'):
    out = []
    for s in signals:
        tkr = s['Ticker'].strip().upper(); side = s['Action'].strip().upper()
        w = float(s['Weight']); cap = float(s['Capital'])
        rows = market.get(tkr, [])
        if not rows:
            out.append({'Ticker':tkr,'Action':side,'RequestedQty':0,'ExecutedQty':0,
                        'AvgFillPrice':0.0,'FinalPrice':0.0,'PNL':0.0})
            continue
        ref_price = rows[0]['Price'] if reference_price_by=='first' else rows[-1]['Price']
        req = int(math.floor((w*cap)/max(ref_price,1e-9)))
        prices = [r['Price'] for r in rows]; vols = [r['Volume'] for r in rows]
        if method.upper()=='VWAP': slices = slice_vwap(req, vols)
        else: slices = slice_twap(req, len(rows))
        fills = cap_by_participation(slices, vols, max_participation)
        exec_qty = sum(fills); avgpx = avg_fill_price(prices, fills)
        fpx = rows[-1]['Price']; pl = pnl(side, avgpx, fpx, exec_qty)
        out.append({'Ticker':tkr,'Action':side,'RequestedQty':req,'ExecutedQty':exec_qty,
                    'AvgFillPrice':round(avgpx,4),'FinalPrice':round(fpx,4),'PNL':round(pl,4)})
    return out

In [6]:
# Example signals (can later replace with data/signals.csv)
signals = [
    {'Ticker':'AAPL','Action':'BUY','Weight':0.4,'Capital':40000},
    {'Ticker':'MSFT','Action':'SELL','Weight':0.2,'Capital':20000},
]

market = load_market_data(market_csv)
results = simulate_execution(signals, market, method='TWAP', max_participation=1.0)
save_execution_results(output_csv, results)
results

[{'Ticker': 'AAPL',
  'Action': 'BUY',
  'RequestedQty': 91,
  'ExecutedQty': 91,
  'AvgFillPrice': 175.5934,
  'FinalPrice': 176.2,
  'PNL': 55.2},
 {'Ticker': 'MSFT',
  'Action': 'SELL',
  'RequestedQty': 12,
  'ExecutedQty': 12,
  'AvgFillPrice': 318.2,
  'FinalPrice': 318.2,
  'PNL': 0.0}]